In [1]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2021-01-17 15:44:07--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  23.2MB/s    in 0.2s    

2021-01-17 15:44:07 (23.2 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [2]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta

import math
import random
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr

from collections import deque

TensorFlow 1.x selected.


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jan 17 15:44:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
data = pd.read_csv("ml-100k/u.data", sep='\t',names="userId,movieId,rating,timestamp".split(",")) 
# u1_data = data[data['userId'].isin(target_users)]
u1_data = data.sort_values(by=['timestamp'],ascending=True).copy()
u1_data

,userId,movieId,rating,timestamp
214,259,255,4,874724710
83965,259,286,4,874724727
43027,259,298,4,874724754
21396,259,185,4,874724781
82655,259,173,4,874724843
...,...,...,...,...
46773,729,689,4,893286638
73008,729,313,3,893286638
46574,729,328,3,893286638
64312,729,748,4,893286638


In [5]:
user_list = data['userId']
user_list = user_list.unique()
test_novelty = pd.DataFrame(user_list, columns = ["userId"])

movie_rated_by_user = data.groupby('movieId')['userId'].count().reset_index(name='userIdCount')
movie_rated_by_user = movie_rated_by_user.set_index('movieId')
movie_rated_by_user

,userIdCount
movieId,
1,452
2,131
3,90
4,209
5,86
...,...
1678,1
1679,1
1680,1


In [6]:
index_list = movie_rated_by_user.index.tolist()
def find_missing(lst):
    max_num = max(lst)
    min_num = min(lst)
    num_range = range(min_num, max_num+1)
    missing_number = list(set(num_range) - set(lst))
     
    return missing_number

In [7]:
what = find_missing(index_list)

if len(what) > 0:
  print(what)
  for i in what:
    movie_rated_by_user.loc[i] = [0]
  movie_rated_by_user.sort_index()

else:
  print('no index missing')

no index missing


In [60]:
test_set = pd.DataFrame(columns=['userId','movieId','rating','timestamp'])
test_set

,userId,movieId,rating,timestamp


In [65]:
for i in range(1,len(u1_data.userId.unique())+1):
  user = u1_data[u1_data['userId']==i]
  total_len = len(user)

  # take the last 20% of the rows as testing
  test_len = round(total_len * 20 / 100)

  # check total len of the items per user
  test_row = user[-test_len:]
  test_set = test_set.append(test_row)

In [77]:
# remove the test data from training data

test_set.drop_duplicates(inplace=True)
test_set

,userId,movieId,rating,timestamp
10508,1,208,5,878542960
83307,1,3,4,878542960
8976,1,12,5,878542960
78171,1,58,4,878542960
9811,1,201,3,878542960
...,...,...,...,...
81005,943,450,1,888693158
92536,943,227,1,888693158
95003,943,230,1,888693158
94914,943,229,2,888693158


In [79]:
train_set = u1_data.drop(test_set.index)
train_set

,userId,movieId,rating,timestamp
214,259,255,4,874724710
83965,259,286,4,874724727
43027,259,298,4,874724754
21396,259,185,4,874724781
82655,259,173,4,874724843
...,...,...,...,...
69176,729,322,4,893286637
70222,729,362,4,893286637
74244,729,300,4,893286638
52134,729,333,4,893286638


In [80]:
print("The total unique user in train set: ", len(train_set['userId'].unique()))
print("The total unique user in test set: ", len(test_set['userId'].unique()))

The total unique user in train set:  943
The total unique user in test set:  943
